In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib
import torch
import matplotlib.pyplot as plt
# from tqdm import tqdm_notebook
%matplotlib inline 
# from google.colab.patches import cv2_imshow
from IPython.display import HTML #imports to play videos
from base64 import b64encode 
import cv2 as cv
#from skimage.measure import compare_ssim
import glob
import time
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import math
import pickle
from functools import partial
from collections import defaultdict

from PIL import Image
from glob import glob

import cv2
import skimage.measure
import albumentations as A
from tqdm.notebook import tqdm 
#from albumentations.pytorch import ToTensor 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models.video import mc3_18, r2plus1d_18

from facenet_pytorch import MTCNN



import sys
import sklearn

import cv2
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt
from torchvision import models, transforms


import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)


device = 'cuda'


DATA_FOLDER = os.getcwd()
TRAIN_SAMPLE_FOLDER = "dfdc_train_part_1"
TEST_FOLDER = "dfdc_train_part_0"


BASE_PATH = os.getcwd()

# define the names of the training, testing, and validation
# directories
TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

REAL = 'REAL'
FAKE = 'FAKE'


IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

# initialize the list of class label names
CLASSES = ["FAKE", "REAL"]


In [2]:
# train metadata

from glob import glob
import json
def get_videos_from_folder(folder_path):
    # List all files in the specified folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if 'metadata.json' in files:
        files.remove('metadata.json')
    full_paths = [os.path.join(folder_path, f) for f in files]
    return full_paths

def get_videos_basenames_from_folder(folder_path):
    # List all files in the specified folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if 'metadata.json' in files:
        files.remove('metadata.json')
    return files

def get_original_with_fakes(root_dir):
    pairs = []
    for json_path in glob(os.path.join(root_dir, "*/metadata.json")):
        with open(json_path, "r") as f:
            metadata = json.load(f)
        for k, v in metadata.items():
            original = v.get("original", None)
            if v["label"] == "FAKE":
                pairs.append((original[:-4], k[:-4] ))

    return pairs


metadata_val = pd.read_json(TEST_FOLDER+ '/metadata.json')
metadata_train = pd.read_json(TRAIN_SAMPLE_FOLDER+ '/metadata.json')
#metadata['aaqaifqrwn.mp4']['label']

def sample_weights(metadata):
    video_fake_real_labels = metadata.iloc[0]
    class_counts = video_fake_real_labels.value_counts()
    class_weights = 1 / class_counts
    return [1/class_counts[i] for i in video_fake_real_labels] # for undersampling
from torch.utils.data import WeightedRandomSampler
train_weights_sampler = WeightedRandomSampler(weights=sample_weights(metadata_train), num_samples=len(metadata_train.iloc[0]), replacement=True)
val_weights_sampler =WeightedRandomSampler(weights=sample_weights(metadata_val), num_samples=len(metadata_val.iloc[0]), replacement=True)

In [3]:
# validation set will be 00
metadata_val = pd.read_json(TEST_FOLDER+ '/metadata.json')
metadata_train = pd.read_json(TRAIN_SAMPLE_FOLDER+ '/metadata.json')
video_fake_real_labels = metadata_train.iloc[0]
real_df = video_fake_real_labels[video_fake_real_labels == 'REAL'].keys()
fake_df = video_fake_real_labels[video_fake_real_labels == 'REAL'].keys()

In [4]:
frames_per_video = 32
mtcnn = MTCNN(margin=20, select_largest=False, factor=0.5, device=device, post_process=True) # post_process=False if want human readable
img_size = 380
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
class ImageTransform:
    def __init__(self, size, mean, std):
        self.data_transform = transforms.Compose([
                transforms.Resize((size, size), interpolation=Image.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])

    def __call__(self, img):
        return self.data_transform(img)



def extract_frames_from_video(path, n_frames=32):
    # Create video reader and find length
    v_cap = cv2.VideoCapture(path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Pick 'n_frames' evenly spaced frames to sample
    if n_frames is None:
        sample = np.arange(0, v_len)
    else:
        sample = np.linspace(0, v_len - 1, n_frames).astype(int)
    # Loop through frames
    faces = []
    frames = []
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
            # Load frame
            success, frame = v_cap.retrieve()
            if not success:
                continue
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    return frames



c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

c:\Users\Sheryl\A

In [5]:
class DeepfakeDataset(Dataset):
    def __init__(self, metadata, device, detector, transform, mode='train'):
        self.device = device
        self.detector = detector
        self.transform = transform
        self.folder_path = TRAIN_SAMPLE_FOLDER if mode=='train' else TEST_FOLDER
        self.video_fake_real_labels = metadata.iloc[0]

    def __len__(self):
        return len(self.video_fake_real_labels)

    def __getitem__(self, idx):
        filename = self.video_fake_real_labels.index[idx]
        label = 0 if self.video_fake_real_labels[filename] == 'REAL' else 1
        path = os.path.join(DATA_FOLDER, self.folder_path, filename)
        img_list = []
        # Movie to Image
        try:
            frames = extract_frames_from_video(path)
        except:
            return [], label
        # Detect Faces
        for frame in frames:
            try:
                _frame = frame[np.newaxis, :, :, :]
                boxes, probs = self.detector.detect(_frame, landmarks=False)
                x = int(boxes[0][0][0])
                y = int(boxes[0][0][1])
                z = int(boxes[0][0][2])
                w = int(boxes[0][0][3])
                image = frame[y-15:w+15, x-15:z+15]
                
                # Preprocessing
                image = Image.fromarray(image)
                image = self.transform(image)
                
                img_list.append(image)

            except Exception as e:
                img_list.append(None)
            
        # Padding None
        img_list = [c for c in img_list if c is not None]
        
        return img_list, label

In [9]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model = EfficientNet.from_name('efficientnet-b1')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))
#model._fc = nn.Linear(in_features=model._fc.in_features, out_features=1)
class EffnetTest(nn.Module):
    def __init__(self):
        super(EffnetTest, self).__init__()
        self.model = EfficientNet.from_name(f"efficientnet-b1")
        self.model._fc = nn.Linear(in_features=1280, out_features=1)
        #self.model._norm_layer = nn.GroupNorm(num_groups=32, num_channels=3)
        
    def forward(self, x):
        x = self.model(x)
        return torch.sigmoid(x).squeeze()
model = EffnetTest()

In [10]:
train_dataset = DeepfakeDataset(metadata_train, device, mtcnn, ImageTransform(img_size, mean, std), mode='train')
train_loader = DataLoader(train_dataset, shuffle=False, sampler=train_weights_sampler, batch_size=1) # process video 1 by 1
val_dataset = DeepfakeDataset(metadata_val, device, mtcnn, ImageTransform(img_size, mean, std), mode='train')
val_loader = DataLoader(val_dataset, shuffle=False, sampler=val_weights_sampler, batch_size=1) # process video 1 by 1

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

loss_history = []
accuracy_history = []


In [ ]:
epochs = 5
model = model.to(device)
#data = data.to(device)  # Move data to GPU if available
for epoch in range(epochs): 
    print('epoch', epoch)
    for data, label in train_loader:
        model.train()  # set the model to training mode
        optimizer.zero_grad()  # clear previous gradients
        outputs=[]
        for face in data:
            face = face.to(device)
            output = model(face)
            outputs.append(output)
        answer = torch.mean(torch.stack(outputs), dim=0)
        loss = loss_fn(answer, label.squeeze().float())
        loss.backward()
       
torch.save(model.state_dict(), 'model.pth')

epoch 0
1
1
1
1
1
1
1
1


In [9]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
model = EfficientNet.from_name('efficientnet-b1')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))
model._fc = nn.Linear(in_features=model._fc.in_features, out_features=1)